In [ ]:
import requests
import json
import pandas as pd

http = requests.Session()

url = f'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={token}&units=metric'
response = http.get(url)

info = json.loads(response.text)
df_main_columns = pd.json_normalize(info).head()
df_weather_columns = pd.json_normalize(info, record_path='weather').add_prefix('weather.')
df_final = pd.concat([df_main_columns, df_weather_columns], axis=1).drop(['weather'], axis=1)
df_final['dt_not_timezoned'] = pd.to_datetime(df_final['dt'], unit='s')
df_final

In [9]:
json_body = [
                {
                    'measurement': 'weather',
                    'tags':
                    {
                        'country': df_final['sys.country'][0],
                        'city': df_final['name'][0],
                        'weather_class': df_final['weather.main'][0]
                    },
                    'time': df_final['dt_not_timezoned'][0].isoformat('T') + 'Z', 
                    'fields': 
                    {
                        'visibility': df_final['visibility'][0],
                        'temperature': df_final['main.temp'][0],
                        'sensation': df_final['main.feels_like'][0],
                        'pressure': df_final['main.pressure'][0],
                        'humidity': df_final['main.humidity'][0],
                        'cloudiness': df_final['clouds.all'][0] 
                    }
                }
            ]

In [10]:
from influxdb_client import InfluxDBClient
from influxdb_client.client.write_api import SYNCHRONOUS
my_token = 'your token'
my_org = 'your org'
my_bucket = 'your bucket'
with InfluxDBClient(url='https://us-east-1-1.aws.cloud2.influxdata.com', token=my_token, org=my_org) as client:
    write_api = client.write_api(write_options=SYNCHRONOUS)
    write_api.write(my_bucket, my_org, json_body )
    client.close()